# My LEEDS 2023

Prepares data for the My LEEDS 2023 page 

In [1]:
from events import (
    load_event_data, filter_by_project, prepare, save_files, count_wards
)

Set output directory

In [2]:
OUT_DIR = '../../../../docs/metrics/events/my-leeds-2023/_data/events'

Read in event data and filter by **My LEEDS 2023** project

In [3]:
data = load_event_data().pipe(filter_by_project, '12 - My LEEDS 2023')

Run the processing and save the files

In [4]:
data = data.pipe(prepare).pipe(save_files, OUT_DIR)

Print a list of events which have no ward, or more than one

In [5]:
data[data.pipe(count_wards) != 1][[
    'airtable_id',
    'event_name',
    'start_date',
    'ward_(from_venue)',
]]

,airtable_id,event_name,start_date,ward_(from_venue)
11,265,My Leeds Event Day 1 (Live Activation),2023-07-08,NaN
507,656,The Gift: digital gift launch,2023-06-19,NaN
571,657,The Gift: Garments,2023-06-17,NaN
687,430,My LEEDS Summer: Alwoodley Festival of Sport a...,2023-07-09,"['Alwoodley', 'Alwoodley', 'Alwoodley', 'Alwoo..."


Print a list of events with no audience recorded

In [6]:
data[data.audience_size == 0][[
    'airtable_id', 'event_name', 'start_date', 'audience_size'
]]

,airtable_id,event_name,start_date,audience_size
11,265,My Leeds Event Day 1 (Live Activation),2023-07-08,0.0
38,560,Pudsey Grant Info Session,2023-02-13,0.0
40,566,Weetwood Grant Info Session 2,2023-02-19,0.0
43,431,My LEEDS Summer: Harewood The BIG My Leeds Sum...,2023-07-09,0.0
50,559,Otley and Yeadon Grant Info Session 2,2023-02-21,0.0
71,565,Weetwood Grant Info Session 1,2023-02-15,0.0
74,428,My LEEDS Summer: Gipton & Harehills,2023-07-08,0.0
91,432,My LEEDS Summer: Ardsley & Robin Hood Celebrat...,2023-07-09,0.0
104,563,Roundhay Grant Info Session 2,2023-02-16,0.0
116,558,Otley and Yeadon Grant Info Session 1,2023-02-20,0.0
